# Purpose

This notebook will find the 50k most active Lichess players (rapid blitz classical), measured by number of games.

It does this by downloading and reading parquet files of raw game data.

It's not practical to download the hundreds of 1GB parquet files; so we will download a certain number and extrapolate from there.

We don't need to know exactly who is the *most* active, just need to find *very* active players.

Then, we will process their games for data to feed in to our chess opening recommender AI model.

# Reason

Originally, we were just collecting data on millions of players without caring who was active or not.

This grew impractical quickly, with a massive local DB. Games processing slowed down exponentially because SQL queries to the local duckdb got so slow due to the db's size.

So I've decided to focus on the 50k most active players instead.

# Method

1. Get names of parquet files
    - We will want to look at games data over the course of a year or so
    - So, we'll download some 1GB raw game data parquet files (number TBD) from each month (there are about 60-70 total per month)
    - To do this, we need the names of those files to get from the HuggingFace API
    - Luckily we already have functionality elsewhere to do this; I'll import it here and update slightly for our needs

2. Download parquet files
    - Once we have the names of the needed files, we'll download them
    - I already have functionality to do this elsewhere, just need to import it here

3. Save game counts
    - Once we have our parquet files downloaded, we'll count the number of games each username has
    - There will be some filters; only Rapid/Blitz/Classical games, only players over 1200 rating
    - Need to decide on a data saving method; probably in a CSV with username/num_games
    - Wherever it's saved it needs to be persistent, as this processing will happen over multiple sessions.

4. Get most active players
    - Finally, we'll analyze our CSV to retrieve the usernames of the 50,000 most active Lichess players
    - Profit


- Note that all data for this will be saved in data/processed/find_most_active_players directory.

## Config

Below, we'll define some important variables that we want to use in this notebook.

I like to keep them in one place at the top.

In [ ]:
from utils.downloading_raw_parquet_data.get_parquet_file_names import (
    get_parquet_file_names,
)

# The year and month we want to download data for

month_for_downloading = 6
year_for_downloading = 2025

max_files_to_download_per_month = 3

## 1. Get file names

Now, we'll get the file names for the month and year we want to download.

In [3]:
file_names_for_month = get_parquet_file_names(year_for_downloading, month_for_downloading)
# print with line separations
print("\n".join(file_names_for_month[:max_files_to_download_per_month]))

data/year=2025/month=06/train-00000-of-00065.parquet
data/year=2025/month=06/train-00001-of-00065.parquet
data/year=2025/month=06/train-00002-of-00065.parquet


## Database Setup

Now, we'll initialize our database.

This is a small database that tracks the number of games played by a Lichess username, as well as which files have been checked so there are no duplicates.

In [4]:
import os
from pathlib import Path
from utils.database.db_utils import get_db_connection
from utils.database.player_game_counts_db_utils import setup_player_game_counts_table

# Define the project root
project_root = Path.cwd().parent

# Initialize the database
DB_PATH = project_root / "data" / "processed" / "find_most_active_players" / "player_game_counts.duckdb"
DB_DIR = DB_PATH.parent

# Ensure the directory exists
if not DB_DIR.exists():
    DB_DIR.mkdir(parents=True)
    print(f"Created directory: {DB_DIR}")

# Ensure the database file exists
if not DB_PATH.exists():
    print(f"Database file {DB_PATH} does not exist. Initializing...")
    con = get_db_connection(str(DB_PATH))
    setup_player_game_counts_table(con)
    con.close()
    print("Database created and initialized.")
else:
    print(f"Database file {DB_PATH} already exists. Skipping initialization.")

Database file /Users/a/Documents/personalprojects/chess-opening-recommender/data/processed/find_most_active_players/player_game_counts.duckdb already exists. Skipping initialization.


## Main Pipeline
Now that we have the names of the files we need, we will do the following for each file name (until we've downloaded our predefined max number of files for the given month)

1. Check our local db to make sure that we haven't already downloaded the file in question
    - If we have, cycle through that month's list of files until we find one that hasn't been downloaded
2. Download the file in question
    - Note that HuggingFace's API is smart enough to not re-download files we already have on our local machine, which saves me a lot of headache when doing this repeatedly for testing.
3. Mark the file as having been processed in our local db to avoid duplicates
4. Process the file, recording each player's num_games in the local db
5. Delete the file
6. Move on to next file unless we've reached our maximum number of files

Goals:

- Thorough logging, especially for how long each step takes, including games per second while processing files
- Each file is about 1.4 million rows so that helps when providing a file ETA
- Also log which file number we're on out of the max total files to download

In [ ]:
import time
from utils.database.player_game_counts_db_utils import is_file_already_downloaded
from utils.downloading_raw_parquet_data.raw_parquet_data_file_downloader import (
    download_single_parquet_file,
)

for file_name in file_names_for_month[:5]:
    start_time = time.time()

    # 1. Check in the local db whether the file has already been processed - if so, move on to next file
    con = get_db_connection(str(DB_PATH))
    is_already_downloaded = is_file_already_downloaded(con, file_name, year_for_downloading, month_for_downloading)
    con.close()

    elapsed_time = time.time() - start_time
    print(f"Checked file {file_name} in {elapsed_time:.2f} seconds.")

    if is_already_downloaded:
        print(f"File {file_name} has already been processed. Skipping.")
        continue

    print(f"File {file_name} has not been processed yet. Processing now.")

    # 2. If it hasn't been processed, download the file
    downloaded_file_path = download_single_parquet_file(
        file_name, year_for_downloading, month_for_downloading
    )


    # 3. Mark the file as processed in the local db. Doing this part first because downloading is cheap and quick, we can just move on to the next file if something goes wrong

    # 4. Process the file, updating player game counts in the local db

    # 5. Delete the file

    # 6. Move on to the next file

Checked file data/year=2025/month=06/train-00000-of-00065.parquet in 0.02 seconds.
File data/year=2025/month=06/train-00000-of-00065.parquet has not been processed yet. Processing now.
Checked file data/year=2025/month=06/train-00001-of-00065.parquet in 0.02 seconds.
File data/year=2025/month=06/train-00001-of-00065.parquet has not been processed yet. Processing now.
Checked file data/year=2025/month=06/train-00002-of-00065.parquet in 0.02 seconds.
File data/year=2025/month=06/train-00002-of-00065.parquet has not been processed yet. Processing now.
Checked file data/year=2025/month=06/train-00003-of-00065.parquet in 0.02 seconds.
File data/year=2025/month=06/train-00003-of-00065.parquet has not been processed yet. Processing now.
Checked file data/year=2025/month=06/train-00004-of-00065.parquet in 0.02 seconds.
File data/year=2025/month=06/train-00004-of-00065.parquet has not been processed yet. Processing now.
